In [14]:
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

from gerrychain import Graph, Partition

import pandas as pd
import geopandas as gpd
import maup
import time

maup.progress.enabled = True

In [15]:
def load_data(file_name):
    start_time = time.time()
    df = gpd.read_file(file_name)
    end_time = time.time()
    print(f'The time to import {file_name} is: {(end_time - start_time) / 60} mins')

    return df

In [16]:
population_df = load_data('./data/co_pl2020_vtd/co_pl2020_vtd.shp')

The time to import ./data/co_pl2020_vtd/co_pl2020_vtd.shp is: 0.7336644132932028 mins


In [17]:
election_df = load_data('./data/election/co_vest_20/co_vest_20.shp')

The time to import ./data/election/co_vest_20/co_vest_20.shp is: 0.018725852171579998 mins


In [18]:
cong_df = load_data('./data/co_cong_adopted_2021/2021_Approved_Congressional_Plan_w_Final_Adjustments.shp')

The time to import ./data/co_cong_adopted_2021/2021_Approved_Congressional_Plan_w_Final_Adjustments.shp is: 0.000578149159749349 mins


In [19]:
cong_df.shape

(8, 6)

In [20]:
print(population_df.columns)
print(election_df.columns)
print(cong_df.columns)

Index(['STATEFP20', 'COUNTYFP20', 'VTDST20', 'GEOID20', 'VTDI20', 'NAME20',
       'NAMELSAD20', 'LSAD20', 'MTFCC20', 'FUNCSTAT20',
       ...
       'P0050002', 'P0050003', 'P0050004', 'P0050005', 'P0050006', 'P0050007',
       'P0050008', 'P0050009', 'P0050010', 'geometry'],
      dtype='object', length=348)
Index(['STATEFP', 'COUNTYFP', 'VTDST', 'NAME', 'PRECINCT', 'G20PREDBID',
       'G20PRERTRU', 'G20PRELJOR', 'G20PREGHAW', 'G20PRECBLA', 'G20PREUWES',
       'G20PREOOTH', 'G20USSDHIC', 'G20USSRGAR', 'G20USSLDOA', 'G20USSODOY',
       'G20USSOEVA', 'G20USSOWRI', 'geometry'],
      dtype='object')
Index(['OBJECTID', 'District', 'Shape_Leng', 'Shape_Le_1', 'Shape_Area',
       'geometry'],
      dtype='object')


In [21]:
print(cong_df)

   OBJECTID  District  Shape_Leng    Shape_Le_1    Shape_Area  \
0         1         1    2.495876  7.814496e+05  4.336640e+09   
1         2         2   11.320395  3.521568e+06  3.218254e+11   
2         3         3   23.731473  7.604425e+06  1.396837e+12   
3         4         4   19.406221  6.234628e+06  9.005584e+11   
4         5         5    3.028264  9.616973e+05  4.109716e+10   
5         6         6    3.414628  1.057001e+06  8.668072e+09   
6         7         7    9.952790  3.209028e+06  2.007174e+11   
7         8         8    4.198182  1.323140e+06  2.876257e+10   

                                            geometry  
0  POLYGON ((3233267.516 1756288.526, 3235896.265...  
1  POLYGON ((2591672.472 2158104.322, 2592908.583...  
2  POLYGON ((2484431.843 2159965.567, 2484825.528...  
3  POLYGON ((3801206.673 2167225.509, 3805671.660...  
4  POLYGON ((3166728.678 1472722.423, 3166738.944...  
5  POLYGON ((3252619.873 1727243.005, 3256436.149...  
6  POLYGON ((3150895.603 1804

In [22]:
district_col_name = "District"

In [25]:
election_df = election_df.to_crs(election_df.estimate_utm_crs())
#election_df = election_df.to_crs('EPSG:4269')

In [ ]:
blocks_to_precincts_assignment = maup.assign(population_df['geometry'].geometry, election_df['geometry'].geometry)

In [23]:
print(maup.doctor(election_df))

100%|█████████████████████████████████████| 3215/3215 [00:02<00:00, 1073.03it/s]
/opt/homebrew/lib/python3.10/site-packages/maup/repair.py:331: UserWarning: Geometry is in a geographic CRS. Results from 'area' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  overlaps = inters[inters.area > 0].make_valid()


There are 1431 overlaps.
There are 3778 holes.
False


In [26]:
repaired_election_df = maup.smart_repair(election_df, snap_precision=8)

Snapping all geometries to a grid with precision 10^( -3 ) to avoid GEOS errors.
Identifying overlaps...


100%|███████████████████████████████████| 11234/11234 [00:03<00:00, 3396.24it/s]


Resolving overlaps...
Assigning order 2 pieces...
Assigning order 3 pieces...
Filling gaps...


Gaps to fill: 100%|███████████████████████████| 528/528 [02:25<00:00,  3.63it/s]


In [27]:
print(maup.doctor(repaired_election_df))

100%|█████████████████████████████████████| 3215/3215 [00:03<00:00, 1057.92it/s]


True


In [28]:
print(maup.doctor(population_df))

100%|█████████████████████████████████████| 3108/3108 [00:02<00:00, 1159.18it/s]
/opt/homebrew/lib/python3.10/site-packages/maup/repair.py:331: UserWarning: Geometry is in a geographic CRS. Results from 'area' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  overlaps = inters[inters.area > 0].make_valid()


True


In [29]:
print(maup.doctor(cong_df))

100%|█████████████████████████████████████████████| 8/8 [00:00<00:00, 91.26it/s]

There are 5 holes.
False


In [30]:
repaired_cong_df = maup.smart_repair(cong_df, snap_precision=8)

Snapping all geometries to a grid with precision 10^( -2 ) to avoid GEOS errors.
Identifying overlaps...


100%|██████████████████████████████████████████| 12/12 [00:00<00:00, 587.57it/s]


Resolving overlaps...
Assigning order 2 pieces...
Filling gaps...


Gaps to simplify: 100%|███████████████████████████| 3/3 [00:00<00:00,  6.78it/s]
Gaps to fill: 0it [00:00, ?it/s]


In [32]:
print(maup.doctor(repaired_cong_df))

100%|█████████████████████████████████████████████| 8/8 [00:00<00:00, 93.03it/s]

True
